In [1]:
# HOME CREDIT DEFAULT RISK COMPETITION
# Ce script est une solution pour la compétition Kaggle "Home Credit Default Risk".
# La plupart des caractéristiques sont créées en appliquant des fonctions (min, max, mean, sum, var) 
# à des tables groupées. Peu de sélection de caractéristiques est effectuée, et le surapprentissage 
# (overfitting) peut être un problème car de nombreuses caractéristiques sont corrélées.
# Idées clés utilisées :
# - Diviser ou soustraire des caractéristiques importantes pour obtenir des taux (ex: annuité et revenu).
# - Données Bureau : créer des caractéristiques spécifiques pour les crédits Actifs et Fermés.
# - Demandes Précédentes : créer des caractéristiques spécifiques pour les demandes Approuvées et Refusées.
# - Modularité : une fonction pour chaque table (sauf bureau_balance et application_test qui sont traitées avec leur table principale).
# - Encodage One-Hot pour les caractéristiques catégorielles.
# Toutes les tables sont jointes au DataFrame principal 'application' via la clé SK_ID_CURR (sauf bureau_balance).

# Mise à jour du kernel original (16/06/2018) :
# - Ajout de la caractéristique "Payment Rate".
# - Suppression de l'index des caractéristiques.
# - Utilisation de KFold standard (non stratifié) pour la validation croisée.

# --- Importation des bibliothèques nécessaires ---
import numpy as np  # Pour les opérations numériques (arrays, etc.)
import pandas as pd  # Pour la manipulation des données (DataFrames)
import gc  # Garbage collector, pour la gestion manuelle de la mémoire
import time  # Pour mesurer le temps d'exécution
import re  # Expressions régulières, utilisées pour nettoyer les noms de colonnes
import lightgbm as lgb  # Bibliothèque pour le modèle LightGBM
from contextlib import contextmanager  # Utilitaires pour les gestionnaires de contexte (ex: timer)
from sklearn.metrics import roc_auc_score, roc_curve  # Métriques d'évaluation
from sklearn.model_selection import KFold, StratifiedKFold  # Outils pour la validation croisée
import matplotlib.pyplot as plt  # Pour la création de graphiques
import seaborn as sns  # Pour des graphiques statistiques améliorés
import warnings  # Pour gérer les avertissements Python

# Ignore les avertissements de type FutureWarning pour un affichage plus propre de la sortie
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# --- Fonctions Utilitaires ---

@contextmanager
def timer(title):
    """
    Gestionnaire de contexte pour chronométrer l'exécution d'un bloc de code.

    Args:
        title (str): Titre à afficher avec le temps d'exécution.
    """
    t0 = time.time()  # Temps de début
    yield  # Exécute le bloc de code à l'intérieur du 'with'
    # Affiche le titre et le temps écoulé en secondes
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def sanitize_lgbm_col_name(col_name):
    """
    Nettoie un nom de colonne pour le rendre compatible avec LightGBM,
    en remplaçant les caractères non alphanumériques (sauf '_') par '_'.

    Args:
        col_name (str): Nom de la colonne à nettoyer.

    Returns:
        str: Nom de colonne nettoyé.
    """
    return re.sub(r'[^A-Za-z0-9_]+', '_', str(col_name))

def one_hot_encoder(df, nan_as_category=True):
    """
    Encode les colonnes catégorielles d'un DataFrame en utilisant One-Hot Encoding.

    Args:
        df (pd.DataFrame): DataFrame à encoder.
        nan_as_category (bool, optional): Si True, traite les NaN comme une catégorie distincte. 
                                          Par défaut à True.

    Returns:
        pd.DataFrame: DataFrame avec les colonnes catégorielles encodées.
        list: Liste des noms des nouvelles colonnes créées par l'encodage.
    """
    original_columns = list(df.columns)  # Sauvegarde des noms de colonnes originaux
    # Sélectionne les colonnes de type 'object' (généralement des chaînes de caractères)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    # Applique pd.get_dummies pour le One-Hot Encoding
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    # Identifie les nouvelles colonnes ajoutées
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


In [3]:
# --- Fonctions de Prétraitement par Fichier de Données ---

def application_train_test(num_rows=None, nan_as_category=False):
    """
    Prétraite les fichiers application_train.csv et application_test.csv.
    Combine les deux, nettoie les données, encode les catégories et crée de nouvelles caractéristiques.

    Args:
        num_rows (int, optional): Nombre de lignes à charger pour chaque fichier (utile pour le débogage).
                                  Par défaut à None (charge tout).
        nan_as_category (bool, optional): Pour one_hot_encoder, indique si les NaN doivent être une catégorie.
                                          Ici, il est False par défaut, contrairement à d'autres fonctions.

    Returns:
        pd.DataFrame: DataFrame combiné et prétraité.
    """
    # Lecture des données d'entraînement et de test
    df = pd.read_csv('./data/application_train.csv', nrows=num_rows)
    test_df = pd.read_csv('./data/application_test.csv', nrows=num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    
    # Concaténation des dataframes train et test pour un traitement uniforme
    df = pd.concat([df, test_df]).reset_index(drop=True) # drop=True évite d'ajouter l'ancien index comme colonne
    
    # Nettoyage de données spécifique
    # Suppression des quelques applications avec CODE_GENDER 'XNA'
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Encodage binaire pour les caractéristiques avec seulement deux catégories attendues
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature]) # factorize assigne 0, 1, ... à chaque catégorie
        
    # Encodage One-Hot pour les autres caractéristiques catégorielles
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # Traitement des valeurs NaN pour DAYS_EMPLOYED: 365243 (valeur sentinelle) est remplacée par NaN
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # Création de nouvelles caractéristiques (Feature Engineering) - souvent des ratios
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']  # % de la vie passé à travailler
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT'] # Ratio revenu / montant du crédit
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS'] # Revenu par personne dans la famille
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL'] # Ratio annuité / revenu
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT'] # Taux de paiement (annuité / crédit)
    
    del test_df  # Libération de la mémoire
    gc.collect() # Appel explicite du garbage collector
    return df

def bureau_and_balance(num_rows=None, nan_as_category=True):
    """
    Prétraite bureau.csv et bureau_balance.csv.
    Effectue des agrégations sur bureau_balance, les joint à bureau,
    puis agrège le résultat par SK_ID_CURR.

    Args:
        num_rows (int, optional): Nombre de lignes à charger.
        nan_as_category (bool, optional): Pour one_hot_encoder.

    Returns:
        pd.DataFrame: DataFrame agrégé contenant des caractéristiques issues de bureau et bureau_balance.
    """
    bureau = pd.read_csv('./data/bureau.csv', nrows=num_rows)
    bb = pd.read_csv('./data/bureau_balance.csv', nrows=num_rows)
    
    # Encodage One-Hot pour les deux dataframes
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Agrégations sur bureau_balance (bb)
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']} # Statistiques de base pour MONTHS_BALANCE
    for col in bb_cat:  # Pour les colonnes one-hot encodées de bb, calculer la moyenne
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations) # Grouper par SK_ID_BUREAU
    # Renommer les colonnes agrégées pour éviter les MultiIndex et clarifier l'origine
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    # Joindre les agrégats de bb à bureau
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True) # SK_ID_BUREAU n'est plus nécessaire après la jointure
    del bb, bb_agg # Libération de mémoire
    gc.collect()
    
    # Agrégations sur bureau (maintenant enrichi avec les infos de bb)
    # Définition des agrégations numériques
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'], # Vient de bb_agg joint
        'MONTHS_BALANCE_MAX': ['max'],  # Vient de bb_agg joint
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'] # Vient de bb_agg joint
    }
    # Définition des agrégations catégorielles (moyenne des colonnes one-hot)
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean'] # Pour les OHE de bureau
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean'] # Pour les OHE de bureau_balance (déjà agrégées une fois)
    
    # Application des agrégations groupées par SK_ID_CURR
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    # Renommage des colonnes avec un préfixe 'BURO_'
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Caractéristiques spécifiques pour les crédits 'Actifs'
    # Sélectionne les crédits actifs (colonne 'CREDIT_ACTIVE_Active' créée par OHE)
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1] 
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations) # Agrégations numériques uniquement
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR') # Jointure à bureau_agg
    del active, active_agg
    gc.collect()
    
    # Caractéristiques spécifiques pour les crédits 'Fermés'
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1] # Sélectionne les crédits fermés
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations) # Agrégations numériques uniquement
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR') # Jointure à bureau_agg
    del closed, closed_agg, bureau
    gc.collect()
    
    return bureau_agg

def previous_applications(num_rows=None, nan_as_category=True):
    """
    Prétraite previous_application.csv.
    Effectue des agrégations par SK_ID_CURR, avec des caractéristiques spécifiques
    pour les demandes approuvées et refusées.

    Args:
        num_rows (int, optional): Nombre de lignes à charger.
        nan_as_category (bool, optional): Pour one_hot_encoder.

    Returns:
        pd.DataFrame: DataFrame agrégé.
    """
    prev = pd.read_csv('./data/previous_application.csv', nrows=num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)
    
    # Remplacement des valeurs sentinelles 365243 par NaN pour plusieurs colonnes de jours
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    
    # Nouvelle caractéristique : ratio montant demandé / montant de crédit accordé
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # Définition des agrégations numériques
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Définition des agrégations catégorielles (moyenne des colonnes one-hot)
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    # Agrégation principale par SK_ID_CURR
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Caractéristiques spécifiques pour les demandes approuvées
    # Sélectionne les demandes approuvées (colonne 'NAME_CONTRACT_STATUS_Approved' créée par OHE)
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations) # Agrégations numériques
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR') # Jointure
    
    # Caractéristiques spécifiques pour les demandes refusées
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1] # Sélectionne les demandes refusées
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations) # Agrégations numériques
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR') # Jointure
    
    del refused, refused_agg, approved, approved_agg, prev # Libération de mémoire
    gc.collect()
    return prev_agg

def pos_cash(num_rows=None, nan_as_category=True):
    """
    Prétraite POS_CASH_balance.csv.
    Effectue des agrégations par SK_ID_CURR.

    Args:
        num_rows (int, optional): Nombre de lignes à charger.
        nan_as_category (bool, optional): Pour one_hot_encoder.

    Returns:
        pd.DataFrame: DataFrame agrégé.
    """
    pos = pd.read_csv('./data/POS_CASH_balance.csv', nrows=num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)
    
    # Définition des agrégations
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'], # Stats sur la balance mensuelle
        'SK_DPD': ['max', 'mean'], # Stats sur les jours de retard de paiement (DPD)
        'SK_DPD_DEF': ['max', 'mean'] # Stats sur les DPD avec tolérance
    }
    for cat in cat_cols: # Moyenne pour les colonnes one-hot encodées
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations) # Agrégation par SK_ID_CURR
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Compter le nombre total d'enregistrements POS CASH par client
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    del pos # Libération de mémoire
    gc.collect()
    return pos_agg
    
def installments_payments(num_rows=None, nan_as_category=True):
    """
    Prétraite installments_payments.csv.
    Crée de nouvelles caractéristiques liées aux paiements et effectue des agrégations.

    Args:
        num_rows (int, optional): Nombre de lignes à charger.
        nan_as_category (bool, optional): Pour one_hot_encoder.

    Returns:
        pd.DataFrame: DataFrame agrégé.
    """
    ins = pd.read_csv('./data/installments_payments.csv', nrows=num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)
    
    # Nouvelles caractéristiques
    # Pourcentage payé par rapport à l'échéance
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    # Différence entre le montant dû et le montant payé
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Jours de retard de paiement (DPD) et jours d'avance de paiement (DBD)
    # S'assure que DPD et DBD sont >= 0
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Définition des agrégations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'], # Nombre de versions d'échéancier uniques
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum'] # Jours auxquels les paiements ont été enregistrés
    }
    for cat in cat_cols: # Moyenne pour les colonnes one-hot
        aggregations[cat] = ['mean']
        
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations) # Agrégation par SK_ID_CURR
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Compter le nombre total d'échéances par client
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    del ins # Libération de mémoire
    gc.collect()
    return ins_agg

def credit_card_balance(num_rows=None, nan_as_category=True):
    """
    Prétraite credit_card_balance.csv.
    Effectue des agrégations larges sur les données de carte de crédit.

    Args:
        num_rows (int, optional): Nombre de lignes à charger.
        nan_as_category (bool, optional): Pour one_hot_encoder.

    Returns:
        pd.DataFrame: DataFrame agrégé.
    """
    cc = pd.read_csv('./data/credit_card_balance.csv', nrows=num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    
    # Suppression de SK_ID_PREV car l'agrégation se fait par SK_ID_CURR
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
    # Agrégation générale (min, max, mean, sum, var) sur toutes les colonnes restantes
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    # Renommage des colonnes (aplatissement du MultiIndex)
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    # Compter le nombre de lignes de carte de crédit par client
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

    # Boucle de conversion de type ajoutée pendant le débogage
    # S'assure que les colonnes potentiellement 'object' après agrégation sont converties en numérique
    for col_name in cc_agg.columns:
        if cc_agg[col_name].dtype == 'object':
            print(f"Dans credit_card_balance, la colonne '{col_name}' est de type object. Tentative de conversion en numérique.")
            cc_agg[col_name] = pd.to_numeric(cc_agg[col_name], errors='coerce') # errors='coerce' met NaN si la conversion échoue
            if cc_agg[col_name].isnull().all() and len(cc_agg[col_name]) > 0:
                print(f"Attention : La colonne '{col_name}' est maintenant entièrement NaN après la conversion.")
    
    del cc # Libération de mémoire
    gc.collect()
    return cc_agg


In [4]:
# --- Fonction de Modélisation ---

def kfold_lightgbm(df, num_folds, stratified=False, debug=False):
    """
    Entraîne un modèle LightGBM en utilisant la validation croisée K-Fold.
    Génère des prédictions out-of-fold et des prédictions pour le jeu de test.
    Calcule et affiche l'importance des caractéristiques.

    Args:
        df (pd.DataFrame): DataFrame complet contenant les données d'entraînement et de test fusionnées et prétraitées.
        num_folds (int): Nombre de plis pour la validation croisée.
        stratified (bool, optional): Si True, utilise StratifiedKFold. Par défaut à False (utilise KFold).
        debug (bool, optional): Si True, des étapes de débogage peuvent être activées (non utilisé dans cette version pour le comportement du modèle).

    Returns:
        pd.DataFrame: DataFrame contenant l'importance des caractéristiques.
    """
    # Séparation des données d'entraînement et de test en fonction de la présence de la colonne 'TARGET'
    train_df = df[df['TARGET'].notnull()].copy()  # .copy() pour éviter les SettingWithCopyWarning
    test_df = df[df['TARGET'].isnull()].copy()
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    
    # Conversion de type forcée (ajoutée pendant le débogage)
    # S'assure que les colonnes sont numériques avant l'entraînement
    print("\nForçage de la conversion des types 'object' en numérique dans train_df et test_df:")
    feats_to_process = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    for current_df in [train_df, test_df]:
        for col in feats_to_process:
            if col in current_df.columns:
                if current_df[col].dtype == 'object':
                    print(f"Dans {('train_df' if current_df is train_df else 'test_df')}, conversion de la colonne '{col}' en numérique.")
                    current_df[col] = pd.to_numeric(current_df[col], errors='coerce')
    print("Fin du forçage de la conversion des types.")

    # Configuration de la validation croisée
    if stratified:
        folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
        
    # Initialisation des tableaux pour stocker les prédictions et l'importance des caractéristiques
    oof_preds = np.zeros(train_df.shape[0])  # Prédictions Out-Of-Fold pour l'ensemble d'entraînement
    sub_preds = np.zeros(test_df.shape[0])   # Prédictions pour l'ensemble de test (sera moyenné sur les plis)
    feature_importance_df = pd.DataFrame()   # Pour stocker l'importance des caractéristiques de chaque pli
    
    # Sélection des caractéristiques à utiliser pour l'entraînement
    # Exclut la cible, les identifiants et un éventuel index hérité
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    # Boucle de validation croisée
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        # Séparation des données pour le pli courant
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # Vérification des types dans train_x juste avant l'appel à clf.fit() (ajoutée pendant le débogage)
        print("\nVérification des types dans train_x juste avant l'appel à clf.fit():")
        object_cols_in_train_x = train_x.select_dtypes(include=['object']).columns
        if len(object_cols_in_train_x) > 0:
            print("Colonnes de type 'object' trouvées dans train_x :")
            for col_check in object_cols_in_train_x:
                print(f"- {col_check}: {train_x[col_check].dtype}")
        else:
            print("Aucune colonne de type 'object' trouvée dans train_x. Tous les types semblent corrects.")
        
        # Initialisation du classifieur LightGBM
        # Paramètres issus d'une optimisation bayésienne (selon le commentaire original du kernel)
        clf = lgb.LGBMClassifier(
            nthread=4,  # Nombre de threads (peut être mis à -1 pour utiliser tous les cœurs)
            n_estimators=10000,  # Nombre maximal d'arbres (l'arrêt anticipé en déterminera le nombre optimal)
            learning_rate=0.02, # Taux d'apprentissage
            num_leaves=34,       # Nombre maximal de feuilles par arbre
            colsample_bytree=0.9497036, # Pourcentage de caractéristiques utilisées par arbre
            subsample=0.8715623,      # Pourcentage d'échantillons utilisés par arbre
            max_depth=8,             # Profondeur maximale de l'arbre
            reg_alpha=0.041545473,   # Régularisation L1
            reg_lambda=0.0735294,  # Régularisation L2
            min_split_gain=0.0222415, # Gain minimal pour effectuer une division
            min_child_weight=39.3259775, # Poids minimal des enfants requis pour une division
            verbose=-1, # Contrôle la verbosité du moteur LightGBM (-1 = erreurs/warnings seulement)
                        # Le paramètre 'silent' est déprécié en faveur de 'verbose'.
        )

        # Définition des callbacks pour l'entraînement
        callbacks = []
        # Arrêt anticipé: arrête l'entraînement si le score sur l'ensemble de validation ne s'améliore pas pendant 'stopping_rounds'
        callbacks.append(lgb.early_stopping(stopping_rounds=200, verbose=True)) 
        # Log des évaluations: affiche les métriques d'évaluation toutes les 'period' itérations
        callbacks.append(lgb.log_evaluation(period=200))
        
        # Entraînement du modèle
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
                eval_metric='auc',  # Métrique d'évaluation utilisée pour l'arrêt anticipé
                callbacks=callbacks)

        # Prédictions sur l'ensemble de validation (pour OOF) et sur l'ensemble de test
        # predict_proba retourne les probabilités pour chaque classe, [:, 1] sélectionne la probabilité de la classe positive (défaut)
        # num_iteration=clf.best_iteration_ utilise le nombre optimal d'arbres trouvé par l'arrêt anticipé
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits # Moyenne des prédictions sur les plis

        # Stockage de l'importance des caractéristiques pour ce pli
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y # Libération de mémoire
        gc.collect()

    # Affichage du score AUC global sur les prédictions OOF
    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    
    # Création du fichier de soumission et affichage de l'importance des caractéristiques (si pas en mode debug)
    if not debug:
        test_df['TARGET'] = sub_preds # Assigne les prédictions moyennées à la colonne TARGET du jeu de test
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index=False) # Sauvegarde
        
    display_importances(feature_importance_df) # Affiche et sauvegarde le graphique des importances
    return feature_importance_df

# --- Fonction d'Affichage de l'Importance des Caractéristiques ---

def display_importances(feature_importance_df_):
    """
    Affiche et sauvegarde un graphique de l'importance des caractéristiques.

    Args:
        feature_importance_df_ (pd.DataFrame): DataFrame contenant les caractéristiques, 
                                               leur importance, et le numéro du pli.
    """
    # Calcule l'importance moyenne pour chaque caractéristique sur tous les plis
    # et sélectionne les 40 plus importantes
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    
    plt.figure(figsize=(8, 10)) # Définit la taille du graphique
    # Crée un barplot avec seaborn
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)') # Titre du graphique
    plt.tight_layout() # Ajuste automatiquement les paramètres du graphique pour un bon affichage
    plt.savefig('lgbm_importances01.png') # Sauvegarde le graphique


In [5]:
# --- Fonction Principale d'Orchestration ---

def main(debug=False):
    """
    Fonction principale qui orchestre le chargement des données, le prétraitement,
    l'ingénierie de caractéristiques, le nettoyage des noms de colonnes et l'entraînement du modèle.

    Args:
        debug (bool, optional): Si True, charge un nombre réduit de lignes pour accélérer l'exécution.
                                Par défaut à False.
    """
    num_rows = 10000 if debug else None # Définit le nombre de lignes à charger en mode debug
    
    # Étape 1: Prétraitement des données d'application (train + test)
    df = application_train_test(num_rows)
    
    # Étapes suivantes: Prétraitement des autres tables et jointure avec df principal
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR') # Jointure sur SK_ID_CURR
        del bureau
        gc.collect()
        
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
        
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
        
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
        
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()

    # Nettoyage des noms de colonnes pour compatibilité avec LightGBM (ajouté pendant le débogage)
    print("\nNettoyage des noms de colonnes pour LightGBM...")
    original_columns = list(df.columns)
    sanitized_columns_map = {col: sanitize_lgbm_col_name(col) for col in original_columns}
    df.rename(columns=sanitized_columns_map, inplace=True)
    
    # Gestion des noms de colonnes dupliqués potentiels après nettoyage
    if len(df.columns) != len(set(df.columns)):
        print("Attention : Des noms de colonnes dupliqués ont été créés après le nettoyage !")
        seen = {}
        new_cols = []
        for col_idx, col_name in enumerate(df.columns): # Utiliser enumerate pour accéder à l'index si besoin, ici juste le nom
            if col_name not in seen:
                seen[col_name] = 1
                new_cols.append(col_name)
            else:
                # Si le nom de colonne est déjà vu, on ajoute un suffixe pour le rendre unique
                # Par exemple, si 'col' apparaît 3 fois, on aura 'col', 'col_1', 'col_2'
                # (correction: le suffixe doit commencer à partir de la 2ème occurrence)
                suffix_num = seen[col_name]
                new_cols.append(f"{col_name}_{suffix_num}") 
                seen[col_name] += 1
        df.columns = new_cols # Assigne la nouvelle liste de noms de colonnes (uniques)
        print("Noms de colonnes dupliqués renommés.")

    print("Noms de colonnes nettoyés.")
    
    # Exécution de l'entraînement et de la prédiction avec LightGBM
    # PARTIE MODELISATION COMMENTEE POUR NE CONSERVER QUE LE FEATURE ENGINEERING
    # with timer("Run LightGBM with kfold"):
    #     feat_importance = kfold_lightgbm(df, num_folds=10, stratified=False, debug=debug)

    # Séparation des dataframes train et test
    train_df = df[df['TARGET'].notnull()].copy()  # .copy() pour éviter les SettingWithCopyWarning
    test_df = df[df['TARGET'].isnull()].copy()

    # Affichage des premières lignes des dataframes train et test
    print(f"\nDATASET TRAIN PRET POUR MODELISATION :\n")
    display(train_df.head(10))
    print(f"\nDATASET TEST PRET POUR MODELISATION :\n")
    display(test_df.head(10))

    print(f"Export en CSV des dataframes train et test...")
    train_df.to_csv("./data/application_train_rdy.csv")
    test_df.to_csv("./data/application_test_rdy.csv")
    print(f"Export terminé.")

# --- Point d'entrée du script ---
if __name__ == "__main__":
    # Ce bloc s'exécute uniquement si le script est lancé directement 
    # (et non importé comme module dans un autre script)
    submission_file_name = "submission_kernel02.csv" # Nom du fichier de soumission
    with timer("Full model run"): # Chronomètre l'exécution complète de la fonction main
        main()

Train samples: 307511, test samples: 48744
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 13s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 13s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 7s
Installments payments df shape: (339587, 26)
Process installments payments - done in 16s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 10s

Nettoyage des noms de colonnes pour LightGBM...
Noms de colonnes nettoyés.

DATASET TRAIN PRET POUR MODELISATION :



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,6.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,100008,0.0,0,0,0,0,99000.0,490495.5,27517.5,454500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,100009,0.0,1,1,0,1,171000.0,1560726.0,41301.0,1395000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,100010,0.0,0,1,0,0,360000.0,1530000.0,42075.0,1530000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,100011,0.0,1,0,0,0,112500.0,1019610.0,33826.5,913500.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,74.0
9,100012,0.0,0,0,0,0,135000.0,405000.0,20250.0,405000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



DATASET TEST PRET POUR MODELISATION :



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
307511,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307512,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307513,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,96.0
307514,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,49.0
307515,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307516,100042,NaN,1,1,0,0,270000.0,959688.0,34600.5,810000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,84.0
307517,100057,NaN,0,1,0,2,180000.0,499221.0,22117.5,373500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307518,100065,NaN,0,0,0,0,166500.0,180000.0,14220.0,180000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307519,100066,NaN,1,0,0,0,315000.0,364896.0,28957.5,315000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,15.0
307520,100067,NaN,1,1,0,1,162000.0,45000.0,5337.0,45000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,87.0


Export en CSV des dataframes train et test...
Export terminé.
Full model run - done in 163s
